In [1]:
import os
import cv2
import time
import numpy as np
import numpy as np
import os
import cv2
import keras
import sklearn
import pandas
from time import time
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.utils import to_categorical
from keras.models import load_model
from keras.layers import *
from keras import layers
from keras import Model
from keras.callbacks import TensorBoard
from keras import optimizers
import matplotlib.pyplot as plt
from keras.applications import *
from sklearn.metrics import classification_report
import time
from keras.models import load_model
from sklearn.metrics import confusion_matrix
from keras.applications.resnet50 import preprocess_input
from keras.applications import *
from keras import regularizers

In [81]:
v_path = r"F:\violent_video_tagging\violence-video-classification\data\new_test_frames\violent\\"
nv_path = r"F:\violent_video_tagging\violence-video-classification\data\new_test_frames\non_violent\\"

# v_path = r"F:\violent_video_tagging\violence-video-classification\data\final\train\violent\\"
# nv_path = r"F:\violent_video_tagging\violence-video-classification\data\final\train\non_violent\\"

In [82]:
violent_frames = []
non_violent_frames = []

for frame in os.listdir(v_path):
    frame = cv2.imread(os.path.join(v_path,frame))
    violent_frames.append(frame)


for frame in os.listdir(nv_path):
    frame = cv2.imread(os.path.join(nv_path,frame))
    non_violent_frames.append(frame)

In [71]:
violent_frames[0].shape

(224, 224, 3)

In [72]:
#Creates sequence 
# SHAPE : (N_SAMPLES,TIMESTEP,WIDTH,HEIGHT,CHANNEL)
def create_seq(violent_frames, non_violent_frames):
    
    print("+++ Creating Sequence... +++")
    
    violent_vid = []
    non_violent_vid = []

    i = 0
    while i < len(violent_frames):
        violent_vid.append(violent_frames[i:i+10])
        i = i+10


    i = 0
    while i < len(non_violent_frames):
        non_violent_vid.append(non_violent_frames[i:i+10])
        i = i+10
        
    violent_vid = np.asarray(violent_vid)
    non_violent_vid = np.asarray(non_violent_vid)
    
    return violent_vid, non_violent_vid


In [73]:
def result(x,y):
    
    print("+++ Generating result... +++")
    
    pred = model.predict(x)
#     print('First prediction:', pred)
    
    score = model.evaluate(x, y,verbose=1)
    print("-----------------------------")
    print('Test loss:', score[0])
    print('Test accuracy:', score[1])
    
    prediction = []
    for p in pred:
        if p>=.5:
            prediction.append(1)
        else:
            prediction.append(0)
    print("-----------------------------")
    print("Classification report")
    print("-----------------------------")
    print(classification_report(y, prediction))
    
    print("-----------------------------")
    print("Confusion Matrix")
    print("-----------------------------")
    conf_mat = confusion_matrix(y, prediction)
    print(conf_mat)
    

In [74]:
def preprocess_lstm(features):
    
    print("+++ Preprocessing data... +++")
    
    violent_features = features[0:30*math.floor(violent_frames.shape[0]/30)]
    non_violent_features = features[violent_frames.shape[0]:violent_frames.shape[0]+30*math.floor(non_violent_frames.shape[0]/30)]


    print("Violent features: ", violent_features.shape)
    print("Non Violent features: ", non_violent_features.shape)


    violent_vid,non_violent_vid = create_seq(violent_features, non_violent_features)
    violent_y, non_violent_y = np.zeros(len(violent_vid)), np.ones(len(non_violent_vid))

    print("Violent Video Seq: ", violent_vid.shape,"Non_violent video Seq: ", non_violent_vid.shape)
    print("Violent Label: ", violent_y.shape, "Non_violent Label: ", non_violent_y.shape)

    try:
        test_x = np.vstack((violent_vid,non_violent_vid))
        test_y = np.append(violent_y, non_violent_y)
    except:
        test_x = non_violent_vid
        test_y = non_violent_y

    print("Total data: ", test_x.shape)
    print("Total target: ", test_y.shape)

    test_x = np.reshape(test_x, (test_x.shape[0],test_x.shape[1],np.prod(test_x.shape[2:])))
    print("(LSTM) After Rehshape: ", test_x.shape)
    
    return test_x,test_y

In [75]:
def feature_extract(violent_frames, non_violent_frames, model):
    
    print("+++ Extracting feature... +++")
    
    violent_frames = np.asarray(violent_frames)
    non_violent_frames = np.asarray(non_violent_frames)

    print ("Before Feature extraction: ")
    print(violent_frames.shape,non_violent_frames.shape)
    all_data = np.vstack((violent_frames,non_violent_frames))
    print("Adding all data: ", all_data.shape)

    #creates feature descriptors
    all_data = all_data.astype('float64')
    desc = preprocess_input(all_data)
    if(model == 'resnet50'):
        loaded_model = resnet50.ResNet50(input_shape=(224,224,3), include_top=False)
    elif(model == 'vgg19'):
        loaded_model = VGG19(input_shape=(224,224,3), include_top=False)
    elif(model == 'vgg16'):
        loaded_model = VGG16(input_shape=(224,224,3), include_top=False)
    else:
        print("Please give model name - 'resnet50', 'vgg19', 'vgg16'")
        
    loaded_model = Model(loaded_model.input,loaded_model.output)
    features = loaded_model.predict(desc,batch_size=10,verbose=1)

    print ("After Feature extraction: ", features.shape)
    
    return features

In [83]:
print("+++ Extracting feature... +++")

violent_frames = np.asarray(violent_frames)
non_violent_frames = np.asarray(non_violent_frames)

# violent_frames = violent_frames.reshape((0,0,0,0))

print ("Before Feature extraction: ")
print(violent_frames.shape,non_violent_frames.shape)


try:
    all_data = np.vstack((violent_frames,non_violent_frames))
except:
    all_data = non_violent_frames
print("Adding all data: ", all_data.shape)

#creates feature descriptors
all_data = all_data.astype('float64')
desc = preprocess_input(all_data)
model = 'resnet50'
if(model == 'resnet50'):
    loaded_model = resnet50.ResNet50(input_shape=(224,224,3), include_top=False)
elif(model == 'vgg19'):
    loaded_model = VGG19(input_shape=(224,224,3), include_top=False)
elif(model == 'vgg16'):
    loaded_model = VGG16(input_shape=(224,224,3), include_top=False)
else:
    print("Please give model name - 'resnet50', 'vgg19', 'vgg16'")

+++ Extracting feature... +++
Before Feature extraction: 
(1142, 224, 224, 3) (1129, 224, 224, 3)
Adding all data:  (2271, 224, 224, 3)


In [84]:

loaded_model = Model(loaded_model.input,loaded_model.output)
features = loaded_model.predict(desc,batch_size=10,verbose=1)

print ("After Feature extraction: ", features.shape)

228/228 [==============================] - 192s 841ms/step
After Feature extraction:  (2271, 7, 7, 2048)


In [85]:
import math
test_x, test_y = preprocess_lstm(features)

+++ Preprocessing data... +++
Violent features:  (1140, 7, 7, 2048)
Non Violent features:  (1110, 7, 7, 2048)
+++ Creating Sequence... +++
Violent Video Seq:  (114, 10, 7, 7, 2048) Non_violent video Seq:  (111, 10, 7, 7, 2048)
Violent Label:  (114,) Non_violent Label:  (111,)
Total data:  (225, 10, 7, 7, 2048)
Total target:  (225,)
(LSTM) After Rehshape:  (225, 10, 100352)


In [65]:
import math
test_x, test_y = preprocess_lstm(features)

+++ Preprocessing data... +++
Violent features:  (120, 7, 7, 2048)
Non Violent features:  (1110, 7, 7, 2048)
+++ Creating Sequence... +++
Violent Video Seq:  (12, 10, 7, 7, 2048) Non_violent video Seq:  (111, 10, 7, 7, 2048)
Violent Label:  (12,) Non_violent Label:  (111,)
Total data:  (123, 10, 7, 7, 2048)
Total target:  (123,)
(LSTM) After Rehshape:  (123, 10, 100352)


In [86]:
model = Sequential()
model.add(LSTM(50, input_shape=(test_x.shape[1],test_x.shape[2]), return_sequences=False, kernel_regularizer=regularizers.l2(0.01)))
model.add(Dense(1,activation='sigmoid'))
model.load_weights('resnet_LSTM.h5')
model.summary()
optimizer = optimizers.Adam(lr=0.001,decay=0.004)
model.compile(loss="binary_crossentropy",optimizer=optimizer,metrics=["accuracy"])

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_7 (LSTM)                (None, 50)                20080600  
_________________________________________________________________
dense_7 (Dense)              (None, 1)                 51        
Total params: 20,080,651
Trainable params: 20,080,651
Non-trainable params: 0
_________________________________________________________________


In [87]:
result(test_x,test_y)

+++ Generating result... +++
8/8 [==============================] - 2s 246ms/step - loss: 0.9603 - accuracy: 0.6044
-----------------------------
Test loss: 0.9602927565574646
Test accuracy: 0.6044444441795349
-----------------------------
Classification report
-----------------------------
              precision    recall  f1-score   support

         0.0       1.00      0.22      0.36       114
         1.0       0.56      1.00      0.71       111

    accuracy                           0.60       225
   macro avg       0.78      0.61      0.54       225
weighted avg       0.78      0.60      0.53       225

-----------------------------
Confusion Matrix
-----------------------------
[[ 25  89]
 [  0 111]]


In [67]:
result(test_x,test_y)

+++ Generating result... +++
4/4 [==============================] - 1s 255ms/step - loss: 0.6966 - accuracy: 0.7480
-----------------------------
Test loss: 0.6965609788894653
Test accuracy: 0.7479674816131592
-----------------------------
Classification report
-----------------------------
              precision    recall  f1-score   support

         0.0       0.28      1.00      0.44        12
         1.0       1.00      0.72      0.84       111

    accuracy                           0.75       123
   macro avg       0.64      0.86      0.64       123
weighted avg       0.93      0.75      0.80       123

-----------------------------
Confusion Matrix
-----------------------------
[[12  0]
 [31 80]]
